In [1]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import time
import os
from selenium import webdriver
import re
import requests
from bs4 import BeautifulSoup

In [2]:
import numpy as np
import pandas as pd
# import patsy

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

In this project, we planned to train a model to predict
users’ rating of a business. The motivation includes that if
we can predict how a user is going to rate a business, then we
can recommend the business that the user is more likely to
rate higher than the others

In [3]:
df18=pd.read_csv("output_final-1.csv")
df18=df18.drop(df18.index[2:61])
df18.head()

,Name,Rating,Type,Address,Average budget,Maximum capacity
0,Le Chalet De La Forêt,4.25,Français,1180 Uccle,180,Not Available
1,La Buvette,4.3,Français,1060 Saint-Gilles,58,Not Available
61,Fanny Thai,4.25,Thaïlandais,1000 Bruxelles,24,Not Available
62,Brinz'L,4.166666667,Français,1180 Uccle,70,Not Available
63,Gou,4.166666667,Français,1150 Woluwe-Saint-Pierre,35,Not Available


In [4]:
df18.columns
na_counts = (df18 == 'Not Available').sum()
print(na_counts)

Name                  0
Rating                0
Type                  0
Address               0
Average budget       70
Maximum capacity    424
dtype: int64


In [5]:
del df18['Maximum capacity']
df18.head()

,Name,Rating,Type,Address,Average budget
0,Le Chalet De La Forêt,4.25,Français,1180 Uccle,180
1,La Buvette,4.3,Français,1060 Saint-Gilles,58
61,Fanny Thai,4.25,Thaïlandais,1000 Bruxelles,24
62,Brinz'L,4.166666667,Français,1180 Uccle,70
63,Gou,4.166666667,Français,1150 Woluwe-Saint-Pierre,35


In [6]:
print(df18.dtypes)

Name              object
Rating            object
Type              object
Address           object
Average budget    object
dtype: object


In [7]:
df18.shape

(424, 5)

In [8]:
df18 = df18.drop(df18[df18.eq('Not available').any(axis=1)].index)
df18 = df18.reset_index(drop=True)

In [9]:
df18 = df18.drop(df18[df18.eq('Not Available').any(axis=1)].index)
df18 = df18.reset_index(drop=True)

Les 'NA' était pas écrits pariels (?)

In [10]:
df18.shape

(341, 5)

In [11]:
df18['Average budget'] = pd.to_numeric(df18['Average budget'], errors='coerce')
df18['Rating'] = pd.to_numeric(df18['Rating'], errors='coerce')
print(df18.dtypes)

Name               object
Rating            float64
Type               object
Address            object
Average budget    float64
dtype: object


In [12]:
print(df18.isnull().sum())


Name              0
Rating            1
Type              0
Address           0
Average budget    0
dtype: int64


In [13]:
df18 = df18.dropna()

In [14]:
X = df18.loc[:, ['Type', 'Address', 'Average budget']]
y=df18['Rating']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
#42 for the answer of all question in universe ;)

In [16]:
train_categories = set(X_train['Type'])
test_categories = set(X_test['Type'])
print("Categories in training set:", train_categories)
print("Categories in test set:", test_categories)

Categories in training set: {'Américain', 'Asiatique', 'Péruvien', 'Vietnamien', 'Chinois', 'Français', 'Indien', 'Malgache', 'Belge', 'Brésilien', 'Africain', 'Thaïlandais', 'Turc', 'Marocain', 'Indonésien', 'Iranien', 'Espagnol', 'Italien', 'Méditerranéen', 'Libanais', 'Du Monde', 'Japonais', 'Éthiopien', 'Européen', 'Slave', 'Grec'}
Categories in test set: {'Libanais', 'Japonais', 'Sud-américain', 'Indien', 'Américain', 'Du Monde', 'Européen', 'Belge', 'Vietnamien', 'Suisse', 'Grec', 'Espagnol', 'Chinois', 'Thaïlandais', 'Italien', 'Français'}


Le problème c'est que certaines peuvent ne pas être dans le training set et dans le test set ==> problème pipeline

In [17]:
print("--- Total ---")
print(y.value_counts(normalize=True))
print("--- Train ---")
print(y_train.value_counts(normalize=True))
print("--- Test ---")
print(y_test.value_counts(normalize=True))

--- Total ---
4.000000    0.394118
3.500000    0.211765
4.500000    0.208824
3.000000    0.064706
2.500000    0.029412
5.000000    0.029412
1.000000    0.014706
2.000000    0.008824
0.500000    0.005882
4.250000    0.005882
4.166667    0.005882
2.250000    0.002941
4.300000    0.002941
3.333333    0.002941
3.833333    0.002941
3.750000    0.002941
4.125000    0.002941
1.500000    0.002941
Name: Rating, dtype: float64
--- Train ---
4.000000    0.382353
3.500000    0.235294
4.500000    0.213235
3.000000    0.051471
2.500000    0.029412
5.000000    0.025735
1.000000    0.014706
2.000000    0.011029
0.500000    0.007353
4.250000    0.007353
2.250000    0.003676
3.750000    0.003676
4.166667    0.003676
1.500000    0.003676
3.333333    0.003676
4.300000    0.003676
Name: Rating, dtype: float64
--- Test ---
4.000000    0.441176
4.500000    0.191176
3.500000    0.117647
3.000000    0.117647
5.000000    0.044118
2.500000    0.029412
3.833333    0.014706
4.125000    0.014706
4.166667    0.01470

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline



In [19]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), ['Average budget']),
        ('categorical', OneHotEncoder(handle_unknown='ignore'), ['Type', 'Address'])
    ])

In [20]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lin_reg', LinearRegression())
])

In [21]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['Average budget']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Type', 'Address'])])),
                ('lin_reg', LinearRegression())])

In [22]:
y_pred = pipeline.predict(X_test)


In [23]:
y_pred = pipeline.predict(X_test)


In [24]:
print(y_pred)

[3.92204358 3.46515826 3.99896622 3.90166342 3.29832473 3.1288391
 3.92635729 3.98724722 3.718868   3.82319167 3.78956689 3.7918168
 3.37666421 4.0159906  3.77156968 3.91524043 3.85596808 3.59312074
 4.33488476 3.93193256 3.43811088 3.88387644 4.38626628 3.9776279
 3.69885765 4.0167532  3.48593669 3.95539417 3.83827086 3.84216684
 3.81239659 4.02407364 3.78268654 3.81356867 4.09661188 3.13995596
 3.66010076 3.86130338 3.35769652 4.04102756 3.87448521 4.08790113
 4.03046324 4.6948322  4.1062241  3.77406896 4.6280169  3.27408027
 4.0628419  3.66971894 3.90870334 3.87998933 3.08551365 3.84715694
 3.33546279 3.95501696 3.88609981 4.13327438 4.15550811 4.10177736
 3.76501417 4.10400073 3.32135737 3.84715694 3.93117384 3.9249824
 2.91592567 4.0415801 ]


In [25]:
mse = mean_squared_error(y_test, y_pred)


In [26]:
mae = mean_absolute_error(y_test, y_pred)


In [27]:
r2 = r2_score(y_test, y_pred)


In [28]:
print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (R2): ", r2)

Mean Squared Error (MSE):  0.4464912499771423
Mean Absolute Error (MAE):  0.5000177500168457
R-squared (R2):  -0.05459258622687391


In [29]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Overall, the MSE and MAE scores are relatively low, which suggests that your model has some predictive power. However, the negative R2 score and explained variance score indicate that your model is not performing well in explaining the variance in the target variable. You may need to consider improving your model or exploring different regression algorithms to achieve better results